# Source Code Analysis

## Initialization

In [1]:
import os
import sys

def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)
        sys.path.append(path)
add_path('/home/jjian03/anaconda3/lib/python3.7/site-packages')
add_path(f'{os.path.abspath(os.path.join(".."))}')


### Load Data

In [2]:
import os
import sys

def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)
        sys.path.append(path)
add_path('/home/jjian03/anaconda3/lib/python3.7/site-packages')
add_path(f'{os.path.abspath(os.path.join(".."))}')


In [3]:
import findspark

findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn


spark = SparkSession.builder. \
                config('spark.app.name', 'Jian Jian - Calculate the number of paper'). \
                config('spark.driver.memory', '20g').\
                config('spark.network.timeout', '600s').\
                config('spark.driver.maxResultSize', '10g').\
                config('spark.executor.memory', '15g').\
                config('spark.kryoserializer.buffer.max', '1g').\
                config('spark.cores.max', '50').\
                getOrCreate()


def load_dataset(spark, path, name):
    return spark.read.parquet(path).registerTempTable(name)

load_dataset(spark, '/user/jjian03/WebResourceQuality.parquet', 'web_resource_quality')
load_dataset(spark, '/user/jjian03/WebResourceQuality_pmid.parquet', 'web_resource_quality_pmid')
load_dataset(spark, '/datasets/MAG_20200403/MAG_Azure_Parquet/mag_parquet/Papers.parquet', 'Paper')
load_dataset(spark, '/user/lliang06/icon/MAG_publication_features.parquet', 'mag')

sc = spark.sparkContext
spark_sql = SQLContext(sc)
print(spark.version)

seed=77
fract=1

2.4.0.cloudera2


In [4]:
df_complete = spark_sql.sql(f'''
    SELECT m.total_num_of_paper_citing, wr.url
    FROM web_resource_quality wr
    JOIN web_resource_quality_pmid wr_doi ON wr.id = wr_doi.id
    JOIN Paper p ON wr_doi.doi = p.doi
    JOIN mag m ON p.paperId = m.paperId
    WHERE wr.label IS NOT NULL
    AND wr.label IN ("0", "1")
    AND isNaN(wr.label) = false
    AND wr.first_appear IS NOT NULL
    AND isNaN(wr.first_appear) = false
    AND lower(wr.url) NOT LIKE "%doi.org%"
''').toPandas()

df_complete.shape

(1965512, 2)

In [5]:
import uuid
import pandas as pd

df_sample = pd.read_csv("untrunc_data_cleaned_url.csv")
df_sample = df_sample.loc[:, ["url"]]
df_sample.loc[:, 'idx'] = df_sample.url.apply(lambda x: uuid.uuid4())
print(df_sample.shape)
print(df_sample.drop_duplicates('idx').shape)

(58910, 2)
(58910, 2)


In [6]:
df = df_sample.merge(df_complete, on='url').drop_duplicates('idx')
df.shape

(58910, 3)

In [7]:
print(df.shape[0])
print(df.total_num_of_paper_citing.sum())

58910
2548458.0
